In [33]:
cd /anaconda3/envs/nlp/lib/python3.7/site-packages

/anaconda3/envs/nlp/lib/python3.7/site-packages


In [34]:
import time
from bs4 import BeautifulSoup
import sys, io
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.proxy import *
import pandas as pd

In [35]:
import MeCab
from os import path
import neologdn
import re
import emoji

pos_list = [10, 11, 31, 32, 34]
pos_list.extend(list(range(36,50)))
pos_list.extend([59, 60, 62, 67])

def create_mecab_list(text):
    mecab_list = []
    mecab = MeCab.Tagger("-Ochasen")
    mecab.parse("")
    # encoding = text.encode('utf-8')
    node = mecab.parseToNode(text)
    while node:
        if len(node.surface) > 1:
            if node.posid in pos_list:
                morpheme = node.surface
                mecab_list.append(morpheme)
        node = node.next
    return mecab_list

def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    cleaned_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return cleaned_text

def clean_text(text):
    text = give_emoji_free_text(text)
    text = neologdn.normalize(text)
    text = create_mecab_list(text)    
    return text

In [36]:
no_of_reviews = 1000
non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

In [37]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True
reviews = pd.DataFrame(columns = ["review", "Author Name", "Review Date", "Review Ratings", 
                                  "Review Body", "Developer Reply"])
temp = {"review": 0, "Author Name": "", "Review Date": "", "Review Ratings": 0, 
        "Review Body": "", "Developer Reply": ""}

def replace_value_with_definition(key_to_find, definition):
    for key in temp.keys():
        if key == key_to_find:
            temp[key] = definition

In [38]:
cd ~/Desktop/chrome

/Users/justinwang/Desktop/chrome


In [39]:
driver = webdriver.Chrome(r"./chromedriver")

In [40]:
wait = WebDriverWait( driver, 10 )

In [41]:
# Append your app store urls here
urls = ["https://play.google.com/store/apps/details?id=com.kddi.android.UtaPass&hl=ja"]

for url in urls:

    driver.get(url)

    page = driver.page_source

    soup_expatistan = BeautifulSoup(page, "html.parser")

    expatistan_table = soup_expatistan.find("h1", class_="AHFaub")

    print("App name: ", expatistan_table.string)

    expatistan_table = soup_expatistan.findAll("span", class_="htlgb")[4]

    print("Installs Range: ", expatistan_table.string)

    expatistan_table = soup_expatistan.find("meta", itemprop="ratingValue")

    print("Rating Value: ", expatistan_table['content'])

    expatistan_table = soup_expatistan.find("meta", itemprop="reviewCount")

    print("Reviews Count: ", expatistan_table['content'])

    soup_histogram = soup_expatistan.find("div", class_="VEF2C")

    rating_bars = soup_histogram.find_all('div', class_="mMF0fd")

    for rating_bar in rating_bars:
        print("Rating: ", rating_bar.find("span").text)
        print("Rating count: ", rating_bar.find("span", class_="L2o20d").get('title'))

    # open all reviews
    url = url + '&showAllReviews=true'
    driver.get(url)
    time.sleep(5) # wait dom ready
    for i in range(1,25):
        try:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # scroll to load other reviews
            time.sleep(2)
            if check_exists_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div[2]/div/content/span'):
                driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div[2]/div/content/span').click()
                time.sleep(2)
        except:
            pass

    page = driver.page_source

    soup_expatistan = BeautifulSoup(page, "html.parser")
    expand_pages = soup_expatistan.findAll("div", class_="d15Mdf")
    counter = 1
    items = []
    
    for expand_page in expand_pages:
        try:
            # print("\n===========\n")
            
            # print("review：" + str(counter))
            review = str(counter)
            
            # print("Author Name: ", str(expand_page.find("span", class_="X43Kjb").text))
            Author_Name = str(expand_page.find("span", class_="X43Kjb").text)
            
            # print("Review Date: ", expand_page.find("span", class_="p2TkOb").text)
            Review_Date = str(expand_page.find("span", class_="p2TkOb").text)
            
            '''
            //didn't find reviewer link
            print("Reviewer Link: ", expand_page.find("a", class_="reviews-permalink")['href'])
            '''
            
            reviewer_ratings = expand_page.find("div", class_="pf5lIe").find_next()['aria-label'];
            reviewer_ratings = reviewer_ratings.split('/')[0]
            reviewer_ratings = ''.join(x for x in reviewer_ratings if x.isdigit())
            # print("Reviewer Ratings: ", reviewer_ratings)
            Reviewer_Ratings = int(reviewer_ratings)
            
            '''
            //didn't find review title
            print("Review Title: ", str(expand_page.find("span", class_="review-title").string))
            '''
            
            # print("Review Body: ", str(expand_page.find("div", class_="UD7Dzf").text))
            Review_Body = str(expand_page.find("div", class_="UD7Dzf").text)
            Review_Body_cleaned = clean_text(Review_Body)
            Review_Body_string = ''.join(Review_Body_cleaned)
            
            developer_reply = expand_page.find_parent().find("div", class_="LVQB0b")
            if hasattr(developer_reply, "text"):
                # print("Developer Reply: " + "\n", str(developer_reply.text))
                Developer_Reply = str(developer_reply.text)
            else:
                # print("Developer Reply: ", "")
                Developer_Reply = ""
            
            counter += 1           
            item = {
                    "review": counter - 1,
                    "Author Name": Author_Name,
                    "Reviewer Ratings": Reviewer_Ratings,
                    "Review Date": Review_Date,
                    "Review Body": Review_Body_string,
                    "Developer Reply": Developer_Reply
                    }
            items.append(item)
                
        except:
            pass
driver.quit()
# df = pd.DataFrame(items, columns=["review", "Author Name", "Reviewer Ratings", "Review Date", "Review Body", "Developer Reply"])

App name:  うたパス
Installs Range:  1,000,000+
Rating Value:  3.464590549468994
Reviews Count:  2344
Rating:  5
Rating count:  None
Rating:  4
Rating count:  None
Rating:  3
Rating count:  None
Rating:  2
Rating count:  None
Rating:  1
Rating count:  None


In [42]:
df = pd.DataFrame(items, columns = ["review", "Author Name", "Review Date", "Reviewer Ratings", 
                                    "Review Body", "Developer Reply"])
df

,review,Author Name,Review Date,Reviewer Ratings,Review Body,Developer Reply
0,1,横田直樹,2019年5月3日,2,歌詞見れるいいずれ誤字あるあとお気に入りプレイリスト開くライブラリ更新リセットれるマジ入れラ...,
1,2,ゆきゆき,2019年5月1日,1,通知切る方法分かりすぎる見る新たアプリ入れ通知切れ判明アプリ開発若者毎日アクティブ新曲買っ聴...,
2,3,s tat,2019年4月30日,1,どうしてもLISMO比べダウンロード反映LISMO動画一覧表示パス分離とにかく使いLISMO...,
3,4,NISHII YASUKO,2019年4月29日,1,以前購入機種だぶっダウンロードれる消す出来機種するダウンロード出来有るガラケー購入スマ出来有り,
4,5,kimi oh,2019年5月4日,1,LISMOライブラリ開けなっ愛着あっLISMO使っ消し下らないうたパスLISMOいらついて最...,
5,6,松尾聖子,2019年5月2日,3,,
6,7,Googleユーザー,2019年4月20日,1,au端末シムフリー端末使用au端末反応鈍くタッチ再生かかるシムフリー端末プレイリスト音楽消え...,
7,8,MA CA,2019年4月18日,2,起動重い音楽聞いバックグラウンドアプリ消し音楽聞けバックグラウンド使え結構愛用もういら,
8,9,佃隆宏,2019年4月26日,4,アルバム音量あげさげ-★ひとつ,
9,10,にしえみ,2019年4月23日,2,購入楽曲購入流れる聞き購入楽曲聞い聞け聞こする操作必要なる,


In [50]:
import numpy as np
df['Review Body'].replace('', np.nan, inplace=True)
df.dropna(subset=['Review Body'], inplace=True)
df.head()

,review,Author Name,Review Date,Reviewer Ratings,Review Body,Developer Reply
0,1,横田直樹,2019年5月3日,2,歌詞見れるいいずれ誤字あるあとお気に入りプレイリスト開くライブラリ更新リセットれるマジ入れラ...,
1,2,ゆきゆき,2019年5月1日,1,通知切る方法分かりすぎる見る新たアプリ入れ通知切れ判明アプリ開発若者毎日アクティブ新曲買っ聴...,
2,3,s tat,2019年4月30日,1,どうしてもLISMO比べダウンロード反映LISMO動画一覧表示パス分離とにかく使いLISMO...,
3,4,NISHII YASUKO,2019年4月29日,1,以前購入機種だぶっダウンロードれる消す出来機種するダウンロード出来有るガラケー購入スマ出来有り,
4,5,kimi oh,2019年5月4日,1,LISMOライブラリ開けなっ愛着あっLISMO使っ消し下らないうたパスLISMOいらついて最...,


In [48]:
cd ~/Desktop

/Users/justinwang/Desktop


In [49]:
df.to_csv("GooglePlayReviews.csv", columns = ["review", "Author Name", "Review Date", "Reviewer Ratings", 
          "Review Body", "Developer Reply"], na_rep = 'NA')